# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
D. Semenov  ->  D. Semenov  |  ['D. Semenov']
J. Shi  ->  J. Shi  |  ['J. Shi']
Arxiv has 63 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2508.19494
extracting tarball to tmp_2508.19494...

 done.


Found 88 bibliographic references in tmp_2508.19494/Main.bbl.
Retrieving document from  https://arxiv.org/e-print/2508.19520


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem[{{Euclid Quick Release Q1}(2025)}]{Q1cite}{Euclid Quick Release Q1}. 2025, \url{https://doi.org/10.57780/esa-2853f3b}
 regex = 
        \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3})(?P<rest>.*)
        
  warnings.warn(str(e))


extracting tarball to tmp_2508.19520... done.
Retrieving document from  https://arxiv.org/e-print/2508.19701


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2508.19520/ms.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'Table2.tex' from 'tmp_2508.19520/Table2.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'Table1.tex' from 'tmp_2508.19520/Table1.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


extracting tarball to tmp_2508.19701...

 done.


Issues with the citations
list index out of range
Retrieving document from  https://arxiv.org/e-print/2508.19715
extracting tarball to tmp_2508.19715... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.19494-b31b1b.svg)](https://arxiv.org/abs/2508.19494) | **Euclid: A machine-learning search for dual and lensed AGN at sub-arcsec separations**  |
|| L. Ulivi, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-08-28*|
|*Comments*| *18 figures*|
|**Abstract**|            Cosmological models of hierarchical structure formation predict the existence of a widespread population of dual accreting supermassive black holes (SMBHs) on kpc-scale separations, corresponding to projected distances < 0".8 at redshifts higher than 0.5. However, close companions to known active galactic nuclei (AGN) or quasars (QSOs) can also be multiple images of the object itself, strongly lensed by a foreground galaxy, as well as foreground stars in a chance superposition. Thanks to its large sky coverage, sensitivity, and high spatial resolution, Euclid offers a unique opportunity to obtain a large, homogeneous sample of dual/lensed AGN candidates with sub-arcsec projected separations. Here we present a machine learning approach, in particular a Convolutional Neural Network (CNN), to identify close companions to known QSOs down to separations of $\sim\,$0".15 comparable to the Euclid VIS point spread function (PSF). We studied the effectiveness of the CNN in identifying dual AGN and demonstrated that it outperforms traditional techniques. Applying our CNN to a sample of $\sim\,$6000 QSOs from the Q1 Euclid data release, we find a fraction of about 0.25% dual AGN candidates with separation $\sim\,$0".4 (corresponding to $\sim$3 kpc at z=1). Estimating the foreground contamination from stellar objects, we find that most of the pair candidates with separation higher than 0".5 are likely contaminants, while below this limit, contamination is expected to be less than 20%. For objects at higher separation (>0".5, i.e. 4 kpc at z=1), we performed PSF subtraction and used colour-colour diagrams to constrain their nature. We present a first set of dual/lensed AGN candidates detected in the Q1 Euclid data, providing a starting point for the analysis of future data releases.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.19701-b31b1b.svg)](https://arxiv.org/abs/2508.19701) | **Time-resolved protoplanetary disk physics in DQ Tau with JWST**  |
|| Á. Kóspál, et al. -- incl., <mark>D. Semenov</mark> |
|*Appeared on*| *2025-08-28*|
|*Comments*| *21 pages, 20 figures, accepted for publication in the A&A*|
|**Abstract**|            Accretion variability is ubiquitous in YSOs. While large outbursts may strongly affect the disk, the effects of moderate bursts are less understood. We aim to study the physical response of the disk around the eccentric binary system DQ Tau to its periodic accretion changes. We organized a multi-wavelength campaign centered on four JWST/MIRI spectra. We targeted three periastrons (high accretion state) and one apastron (quiescence). We used optical and near-infrared spectroscopy and photometry to measure how the accretion luminosity varies. We decomposed the multi-epoch SEDs into stellar, accretion, and rim components. We fitted the solid-state features using various opacity curves and the molecular features using slab models. We find the inner disk of DQ Tau to be highly dynamic. The temperature, luminosity, and location of the inner dust rim vary in response to the movement of stars and the L_acc variations. This causes variable shadowing of the outer disk, leading to an anti-correlation between the rim temperature and the strength of the silicate feature. The dust mineralogy remains constant, dominated by large amorphous olivine and pyroxene grains, with smaller fractions of crystalline forsterite. The excitation of CO (1550-2260 K), HCN (880-980 K), and hot H2O (740-860 K) molecules as well as the luminosity of the [NeII] line correlate with the accretion rate, while the warm (650 K) and cold (170-200 K) H2O components are mostly constant. CO emission, originating from a hot (>1500 K) region likely within the dust sublimation radius, is most sensitive to L_acc changes. In comparison with other T Tauri disks, DQ Tau is highly C-poor and displays moderately inefficient pebble drift. We conclude that even moderate accretion rate changes affect the thermal structure in the planet-forming disk regions on short timescales, providing a crucial benchmark for understanding disk evolution.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.19520-b31b1b.svg)](https://arxiv.org/abs/2508.19520) | **Sensitive Constraints on Coherent Radio Emission from Five Isolated White Dwarfs**  |
|| L. Zhang, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-08-28*|
|*Comments*| **|
|**Abstract**|            Coherent, periodic radio emission from pulsars has been widely interpreted as evidence of neutron stars as strongly magnetized compact objects. In recent years, radio pulses have also been detected from white dwarfs (WDs) in tight binary systems, raising the question of whether isolated WDs could similarly host pulsar-like emission. We conducted the most sensitive search to date for coherent radio signals from five isolated, rapidly rotating, and magnetized WDs, using the Five-hundred-meter Aperture Spherical radio Telescope (FAST), the Green Bank Telescope (GBT), and the Australia Telescope Compact Array (ATCA). No pulsed or continuum radio emission was detected down to $\mu$Jy levels. These non-detections place the most stringent observational constraints yet on the existence of isolated WD pulsars. Our results suggest that either such emission is intrinsically weak, narrowly beamed, or requires binary-induced magnetospheric interactions absent in solitary systems. Comparison with the known radio-emitting WDs highlights the critical role of companion interaction in enabling detectable emission. This work expands on prior surveys by targeting sources with the most favorable physical conditions for WD pulsar-like activity and employing highly sensitive, targeted observations. Future observations with next-generation facilities such as the SKA will be essential to explore fainter or sporadic emission from massive, magnetic WDs and to investigate their potential as compact radio transients further.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.19715-b31b1b.svg)](https://arxiv.org/abs/2508.19715) | **Meridional Circulation II: A Unified Mechanism for Lithium Depletion in Solar Analogs and the Lithium Dip in Mid-F Cluster Stars**  |
|| X. Li, et al. -- incl., <mark>J. Shi</mark> |
|*Appeared on*| *2025-08-28*|
|*Comments*| *Accepted in Astrophysical Journal*|
|**Abstract**|            The behavior of lithium (Li) in Population I main sequence stars challenges standard stellar theory. Two phenomena stand out: the solar Li problem which extends to Li depletion in solar analogs and the Li dip observed in mid-F stars within open clusters. Building on the meridional circulation-driven radial mixing framework previously developed to explain Li-enriched red clump stars, we explore its relevance to Li depletion on the main sequence. First, our models reproduce the observed $A(\text{Li})$-Age correlation in solar analogs. Through detailed isochrone analysis, we find good agreement between the simulated and observed $A(\text{Li})$-$T_{\text{eff}}$ relationships within the solar analog parameter space. However, the predicted solar Li abundance ($\sim 1.5\,\text{dex}$) is still higher than current solar measurements. Second, our models partially explain the Li dip phenomenon in mid-F cluster stars. The models accurately reproduce Li distributions on the cool side of the Li dip in most clusters and capture the Li behaviors on the hot side observed in systems like the Hyades. However, we identify limitations in the models' ability to fully reproduce the dip morphology, particularly due to the rotation velocity distribution of sample stars in this temperature zone.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Max' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2508.19494/./Images/phot_candidate2.png', 'tmp_2508.19494/./Images/hist_result1.png', 'tmp_2508.19494/./Images/close_sources2.png']
copying  tmp_2508.19494/./Images/phot_candidate2.png to _build/html/
copying  tmp_2508.19494/./Images/hist_result1.png to _build/html/
copying  tmp_2508.19494/./Images/close_sources2.png to _build/html/
exported in  _build/html/2508.19494.md
    + _build/html/tmp_2508.19494/./Images/phot_candidate2.png
    + _build/html/tmp_2508.19494/./Images/hist_result1.png
    + _build/html/tmp_2508.19494/./Images/close_sources2.png
found figures ['tmp_2508.19701/./light_dqtau.png', 'tmp_2508.19701/./temperature.png', 'tmp_2508.19701/./nicole2.png']
copying  tmp_2508.19701/./light_dqtau.png to _build/html/
copying  tmp_2508.19701/./temperature.png to _build/html/
copying  tmp_2508.19701/./nicole2.png to _build/html/
exported in  _build/html/2508.19701.md
    + _build/html/tmp_2508.19701/./light_dqtau.png
    + _build/html/tmp_2508.19701/./temperatur

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]$
$\newcommand\aa{@@strip}$</div>



<div id="title">

# $\Euclid$: A machine-learning search for dual and lensed AGN at sub-arcsec separations$\thanks{This paper is published on behalf of the Euclid Consortium.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2508.19494-b31b1b.svg)](https://arxiv.org/abs/2508.19494)<mark>Appeared on: 2025-08-28</mark> -  _18 figures_

</div>
<div id="authors">

L. Ulivi, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** Cosmological models of hierarchical structure formation predict the existence of a widespread population of dual accreting supermassive black holes (SMBHs) on kiloparsec-scale separations, corresponding to projected distances $<$ $\ang{;;0.8}$ at redshifts higher than 0.5. However, close companions to known active galactic nuclei (AGN) or quasars (QSOs) can also be multiple images of the object itself, strongly lensed by a foreground galaxy, as well as foreground stars in a chance superposition. Thanks to its large sky coverage, sensitivity, and high spatial resolution, $\Euclid$ offers a unique opportunity to obtain a large, homogeneous sample of dual/lensed AGN candidates with sub-arcsec projected separations.   Here we present a machine learning approach, in particular a Convolutional Neural Network (CNN), to identify close companions to known QSOs down to separations of $\sim $ $\ang{;;0.15}$ , comparable to the $\Euclid$ VIS point spread function (PSF). We investigated the performance of the CNN trained on a large sample of simulated single and dual AGN.    We studied the effectiveness of the CNN in identifying dual AGN  and demonstrated that it outperforms traditional techniques such as \texttt{IRAFStarFinder} . Applying our CNN to a sample of $\sim $ 6000 QSOs from the Q1 $\Euclid$ data release (covering $\SI{63.1}{\deg\squared}$ ), we find a fraction of about 0.25 \% dual AGN candidates with separation $\sim $  $\ang{;;0.4}$ (corresponding to $\sim $ 3 kpc at $z $ =1). Estimating the foreground contamination from stellar objects, we find that most of the pair candidates with separation higher than $\ang{;;0.5}$ are likely contaminants, while below this limit, contamination is expected to be less than 20 \% .  For objects at higher separation (> $\ang{;;0.5}$ , i.e. 4 kpc at $z $ =1), we performed PSF subtraction and used colour-colour diagrams to constrain their nature. We present a first set of dual/lensed AGN candidates detected in the Q1 $\Euclid$ data, providing a starting point for the analysis of future data releases.

</div>

<div id="div_fig1">

<img src="tmp_2508.19494/./Images/phot_candidate2.png" alt="Fig17" width="100%"/>

**Figure 17. -** Three-colour composite images (\IE, \JE, \HE) of dual AGN candidates derived from MER photometry or PSF subtraction. Systems enclosed in red boxes are those where both sources lie within the AGN region in both photometric methods. The black stars mark the positions of the input sources from the MER catalogue. The size of each cutout is $\ang{;;3} \times \ang{;;3}$. (*fig:phot_cand*)

</div>
<div id="div_fig2">

<img src="tmp_2508.19494/./Images/hist_result1.png" alt="Fig7" width="100%"/>

**Figure 7. -** Comparison between the number of dual candidates found by CNN and the expected contaminants. The number is normalized by the total number of AGN considered after the cuts (5993). Dual AGN candidates found by CNN in bins of separation (in black), number of expected contaminants classified dual by the CNN once we added a primary source without (purple) and with (blue) host galaxy, and
    false positives expected from simulations (in green). (*fig:contamination*)

</div>
<div id="div_fig3">

<img src="tmp_2508.19494/./Images/close_sources2.png" alt="Fig16" width="100%"/>

**Figure 16. -** Dual AGN candidates found by the network with separations between \ang{;;0.15} and \ang{;;0.45}. The number of objects in this separation range exceeds the number expected from foreground objects and false positives caused by clumpy galaxy structure, making these candidates more reliable than those at larger separations. The size of each cutout is $\ang{;;3} \times \ang{;;3}$, and the flux is shown in logarithmic scale.    (*fig:Closesources*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2508.19494"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\linenumbers}$
$\newcommand{\nolinenumbers}$
$\newcommand\natexlab{#1}$</div>



<div id="title">

# Time-resolved protoplanetary disk physics in DQ Tau with JWST

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2508.19701-b31b1b.svg)](https://arxiv.org/abs/2508.19701)<mark>Appeared on: 2025-08-28</mark> -  _21 pages, 20 figures, accepted for publication in the A&A_

</div>
<div id="authors">

Á. Kóspál, et al. -- incl., <mark>D. Semenov</mark>

</div>
<div id="abstract">

**Abstract:**            Accretion variability is ubiquitous in YSOs. While large outbursts may strongly affect the disk, the effects of moderate bursts are less understood. We aim to study the physical response of the disk around the eccentric binary system DQ Tau to its periodic accretion changes. We organized a multi-wavelength campaign centered on four JWST/MIRI spectra. We targeted three periastrons (high accretion state) and one apastron (quiescence). We used optical and near-infrared spectroscopy and photometry to measure how the accretion luminosity varies. We decomposed the multi-epoch SEDs into stellar, accretion, and rim components. We fitted the solid-state features using various opacity curves and the molecular features using slab models. We find the inner disk of DQ Tau to be highly dynamic. The temperature, luminosity, and location of the inner dust rim vary in response to the movement of stars and the L_acc variations. This causes variable shadowing of the outer disk, leading to an anti-correlation between the rim temperature and the strength of the silicate feature. The dust mineralogy remains constant, dominated by large amorphous olivine and pyroxene grains, with smaller fractions of crystalline forsterite. The excitation of CO (1550-2260 K), HCN (880-980 K), and hot H2O (740-860 K) molecules as well as the luminosity of the [NeII] line correlate with the accretion rate, while the warm (650 K) and cold (170-200 K) H2O components are mostly constant. CO emission, originating from a hot (>1500 K) region likely within the dust sublimation radius, is most sensitive to L_acc changes. In comparison with other T Tauri disks, DQ Tau is highly C-poor and displays moderately inefficient pebble drift. We conclude that even moderate accretion rate changes affect the thermal structure in the planet-forming disk regions on short timescales, providing a crucial benchmark for understanding disk evolution.         

</div>

<div id="div_fig1">

<img src="tmp_2508.19701/./light_dqtau.png" alt="Fig12" width="100%"/>

**Figure 12. -** Light curves (top) and accretion luminosity (bottom) of DQ Tau. Vertical dashed lines mark the epochs of periastrons. The solid vertical lines in color show the epochs of the JWST/MIRI observations. (*fig:light*)

</div>
<div id="div_fig2">

<img src="tmp_2508.19701/./temperature.png" alt="Fig7" width="100%"/>

**Figure 7. -** Temperature of the dust rim and of various gas components (top), accretion luminosity and the luminosity of the [Ne II] line at 12.81$ \mu$m (middle), dust rim radius, peak flux of the continuum-subtracted 10$ \mu$m silicate feature, and the distance of the primary (black) and secondary (gray) from the barycenter (bottom). Error bars smaller than the symbol size are not plotted. (*fig:temperature*)

</div>
<div id="div_fig3">

<img src="tmp_2508.19701/./nicole2.png" alt="Fig6" width="100%"/>

**Figure 6. -** Line luminosity ratios as a function of millimeter dust size for the JDISC sample (black dots, from \citealt{arulanantham2025}) and for DQ Tau (red symbols). (*fig:nicole*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2508.19701"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

126  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
